# Controlling With Noisy Measurements

In this exercise, we'll control the coaxial drone using estimated measurements obtained by sensors. 
For convenience, we will designate the $x_t$ the true state vector of the drone which we should like to know. $z_t$ the observation of measurement of the certain parameters (in this case altitude). $\hat{x}_t$ estimated for the state which we would like to know. 

In [1]:
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

import numpy as np 
import math
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import jdc
from ipywidgets import interactive
from CoaxialDrone import CoaxialCopter
from PIDcontroller import PIDController_with_ff
from PathGeneration import flight_path
from DronewithPIDControllerParameters import DronewithPID

pylab.rcParams['figure.figsize'] = 10, 10

In [ ]:
pylab.rcParams['figure.figsize'] = 10, 10

The IMU in this notebook estimates the altitude with a weighted average defined as:

$$
\hat{z}_{t} = \alpha \hat{z}_{t-1} + (1-\alpha)z_t
$$

**NOTE:** For this exercise we do not modify the velocity as it is obtained by integrating the acceleration over time and we do not measure it directly. 

In [2]:
class IMU:
    def __init__(self,
                 z_hat, # initial estimated value
                 alpha  # alpha value how fast to update the estimated value
                ):
        '''
        Initializing the IMU object with initial altitude estimation and the alpha value for the exponential averaging. 
        '''
        self.z_hat = z_hat
        self.alpha = alpha
        
    def measure(self, 
                z,                 # True altitude
                sigma = 0.01       # Error sigma value 
               ):
        '''
        Simulating the sensor measurement
        '''
        # TODO: Simulate the measurement of the altitude by adding the error associated
        # with the measurement to the true method.
        return z + np.random.normal(0.0, sigma)


    def estimate(self, z_t):
        '''
        Estimates the drone altitude using the weighted average method. 
        '''
        # TODO: Estimate the drone altitude using the weighted average method. 
        self.z_hat = (1 - self.alpha) * z_t + self.alpha * self.z_hat
        return self.z_hat

Let us consider a case when at $t=0$ we will ask drone in change the altitude by 1 meter from the stationary position represented as $z=0$ and $\dot{z}=0$. Thus we will generate the desired path with just $z=-1$ for all time $t$. 

In [3]:
total_time = 10.0   # Total Flight time 
dt = 0.01           # A time interval between measurements 

t, z_path, z_dot_path, z_dot_dot_path =  flight_path(total_time, dt,'constant' )

Let's compare the path executed by the drone when we are using the measured altitude to control the controller instead of the actual value which we assumed that was magically given to us.

In [4]:
FlyingDrone = DronewithPID(z_path, z_dot_path, z_dot_dot_path, t, dt, IMU)

In [5]:
interactive_plot = interactive(FlyingDrone.PID_controler_with_measured_values, 
                               k_p=(5.0, 35.0, 1),
                               k_d=(0.0, 10, 0.5), 
                               k_i=(0.0, 10, 0.5), 
                               mass_err =(0.7, 1.31, 0.01),
                               sigma=(0.0, 0.1, 0.001))
output = interactive_plot.children[-1]
output.layout.height = '800px'
interactive_plot

interactive(children=(FloatSlider(value=20.0, description='k_p', max=35.0, min=5.0, step=1.0), FloatSlider(value=5.0, description='k_d', max=10.0, step=0.5), FloatSlider(value=5.0, description='k_i', max=10.0, step=0.5), FloatSlider(value=1.0, description='mass_err', max=1.31, min=0.7, step=0.01), FloatSlider(value=0.05, description='sigma', max=0.1, step=0.001), Checkbox(value=False, description='use_measured_height'), Output(layout=Layout(height='800px'))), _dom_classes=('widget-interact',))

In this section, we will use the estimated value of the altitude based on the averaging to control the drone instead of relying only on the last measurement value.

In [6]:
interactive_plot = interactive(FlyingDrone.PID_controler_with_estimated_values, 
                               k_p=(5.0, 35.0, 1),
                               k_d=(0.0, 10, 0.5), 
                               k_i=(0.0, 10, 0.5), 
                               mass_err =(0.7, 1.31, 0.01),
                               sigma = (0.0, 0.1, 0.001),
                               alpha = (0.51, 0.99, 0.01))
output = interactive_plot.children[-1]
output.layout.height = '800px'
interactive_plot

A Jupyter Widget

# Questions:
---
* Is the magnitude of the steady-state error higher or lower when using the weighted average compared to the using the directly measured values?
* Does the drone take shorter or longer time to converge at the desired altitude using the weighted averaging for altitude estimation relative to using the direct altitude measurement?
---